In [1]:
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai

     |████████████████████████████████| 15.5 MB 790 kB/s eta 0:00:01
     |████████████████████████████████| 174 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 907 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 898 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 902 kB/s eta 0:00:01
     |████████████████████████████████| 177 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 337 kB 940 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 322 kB/s eta 0:00:01
     |████████████████████████████████| 385 kB 477 kB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 1.5 MB/s eta 0:00:01    |█████████████████████████▌      | 9.0 MB 1.1 

In [2]:
%pip install llama-index

     |████████████████████████████████| 2.0 MB 464 kB/s eta 0:00:01
     |████████████████████████████████| 154 kB 612 kB/s eta 0:00:01
     |████████████████████████████████| 295 kB 555 kB/s eta 0:00:01
     |████████████████████████████████| 147 kB 604 kB/s eta 0:00:01
You should consider upgrading via the '/Users/nestorxyz/coding/learn/ai/learn-ai/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.2)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

/Users/nestorxyz/coding/learn/ai/learn-ai/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader("../data/paul").load_data()

In [12]:
from llama_index.core import Settings

# initialize settings (set chunk size)
Settings.chunk_size = 1024
nodes = Settings.node_parser.get_nodes_from_documents(documents)

In [13]:
from llama_index.core import StorageContext

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [15]:
from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex

summary_index = SummaryIndex(nodes, storage_context=storage_context)


Retrying llama_index.embeddings.openai.base.get_embeddings in 0.9421026537416163 seconds as it raised APIConnectionError: Connection error..


APIConnectionError: Connection error.

In [18]:
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [16]:
list_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

NameError: name 'vector_index' is not defined

In [ ]:
from llama_index.core.tools import QueryEngineTool


list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description=(
        "Useful for summarization questions related to Paul Graham eassy on"
        " What I Worked On."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from Paul Graham essay on What"
        " I Worked On."
    ),
)